### ****** 필독 !! 실습과제 주의사항 ******

- 각자의 "수강생 번호" 확인 (아이캠퍼스 공지) 
- 제출하는 실습과제 파일에 "수강생 번호"를 기준으로 작성 (이름, 학번 등 작성X)
- 각 실습과제에 대한 구체적인 파일 이름은 매번 개별 안내 
  (수강생 번호 123번 학생 과제파일 예시 : 'ST123_HW4_1' )
  
- 과제 파일이름 양식 지키지 않을 시 감점!

# [실습 과제 4]

## 1. ExampleData의 센서 데이터 3가지에 대하여 STFT 진행
#### >>>>>> STFT의 Parameter 중 <font color=red>nperseg는 50</font>으로 변경
#### >>>>>> 3가지의 Spectrogram을 DataFrame형태로 저장
#### >>>>>> 저장된 파일 이름 : <font color=blue>ST(수강생 번호)_Acc // ST(수강생 번호)_Vol // ST(수강생 번호)_Cur (예시 : 'ST00_Acc')</font>

## 2. CNN 모델 구조 설계
#### >>>>>> [주의!] <font color=red>1번 문제의 Spectrogram 결과</font>가 아닌 <font color=blue>실습데이터</font>를 이용
#### >>>>>> Layer 순서 : Convolution - MaxPooling - Convolution - MaxPooling - Convolution - GlobalAveragePooling - Output
#### >>>>>> Convolution Layer 1 : 5 by 5의 filter 4개를 2칸씩 이동하며 특징 추출 (padding 사용)
#### >>>>>> MaxPolling   Layer 1 : 2 by 2 pool을 2칸씩 이동하며 subsampling
#### >>>>>> Convolution Layer 2 : 5 by 5의 filter 16개를 1칸씩 이동하며 특징 추출 (padding 사용 X)
#### >>>>>> MaxPolling   Layer 2 : 2 by 2 pool을 2칸씩 이동하며 subsampling
#### >>>>>> Convolution Layer 3 : 3 by 3의 filter 64개를 1칸씩 이동하며 특징 추출 (padding 사용)
#### >>>>>> Output Layer : Output은 정상 / 고장 두 개의 클래스로 출력
#### >>>>>> model.summary() 함수를 이용해 출력된 값을 캡처한 이미지 파일 제출 (이미지 파일 확장자는 jpg 또는 png)
#### >>>>>> 이미지 파일 이름 : <font color=blue>ST(수강생 번호)_summary  (예시 : 'ST00_summary.jpg'  //  'ST0_summary.png')</font>

## ***** 3가지의 Spectrogram과 이미지 파일을 하나의 zip파일로 압축하여 제출
### >>> 압축파일 이름 :  <font color=blue>ST(수강생 번호)_HW4 (예시 : 'ST00_HW4.zip'  //  'ST0_HW4.zip')</font>
### >>> 압축파일 이름 중 'ST', 'HW' 등 영어는 모두 대문자

.

.

.

In [5]:
def CNN_model(input_data):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(4, kernel_size=(5,5), strides=(2, 2), padding='same', activation='relu',
                                  input_shape=(input_data.shape[1],input_data.shape[2],input_data.shape[3]))) # Convolution layer 1
    
    model.add(keras.layers.MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))                                  # Pooling layer 1
    
    model.add(keras.layers.Conv2D(16, kernel_size=(5,5), strides=(1, 1), activation='relu'))   # Convolution layer 2
    
    model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2, 2)))                                 # Pooling layer 2
    
    model.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=(1, 1), padding = 'same', activation='relu'))  # Convolution layer 3
    
    model.add(keras.layers.GlobalAveragePooling2D())                                                          # Global Average Pooling
    
    model.add(keras.layers.Dense(units = 2, activation='softmax'))                                            # Output Layer

    
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    
    return model

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import tensorflow as tf
from tensorflow import keras

Fold       = 5
Fs         = 12800  # Sampling Frequency
NoOfSensor = 1


# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/CNN_Training_Fold%d'%(i+1)
    path2 = './K_FoldData/CNN_Validation_Fold%d'%(i+1)
    c1 = 'temp_Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'temp_Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label_forANN'  , sep=",", header=None))
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label_forANN', sep=",", header=None))
    
    
# 전체 학습용 데이터
temp_Training_All  = np.array(pd.read_csv('./K_FoldData/CNN_Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None))


# STFT test

f,t,Sxx = signal.spectrogram(temp_Training_All, Fs, nperseg=78)

# Data Reshape for CNN
Training_All = Sxx.reshape(Sxx.shape[0], Sxx.shape[1], Sxx.shape[2], NoOfSensor)

for i in range(Fold):
    s1 = 'f,t,Sxx1 = signal.spectrogram(temp_Training_Fold%d, Fs, nperseg=78)'   %(i+1)
    s2 = 'f,t,Sxx2 = signal.spectrogram(temp_Validation_Fold%d, Fs, nperseg=78)' %(i+1)
    exec(s1)
    exec(s2)
    
    s3 = 'Training_Fold%d   = Sxx1.reshape(Sxx1.shape[0], Sxx1.shape[1], Sxx1.shape[2], NoOfSensor)' %(i+1)
    s4 = 'Validation_Fold%d = Sxx2.reshape(Sxx2.shape[0], Sxx2.shape[1], Sxx2.shape[2], NoOfSensor)' %(i+1)
    exec(s3)
    exec(s4)



learningRate  = 0.0001
iteration     = 1000


In [9]:
CNN_model(Training_Fold1).summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 20, 20, 4)         104       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 10, 10, 4)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 6, 6, 16)          1616      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 3, 3, 16)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 64)          9280      
_________________________________________________________________
global_average_pooling2d_3 ( (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                